In [1]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, DatetimeTickFormatter, Div
from bokeh.io import output_file, show
import pandas as pd
import numpy as np

from bokeh.plotting import figure

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (14,10)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
#Settings for plotting
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

def cmaptohex(tup):
    (r,g,b,a) = tup
    return "#{:02x}{:02x}{:02x}".format(int(r*255),int(g*255),int(b*255))
    
import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
locale.setlocale(locale.LC_ALL,"Danish")

# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

'Danish_Denmark.1252'

# Load data 

In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd()[:-5] + ssidatapath
dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')
dfCase = pd.DataFrame(columns=dfAdm.columns)

In [3]:
for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfAdm.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    casedatadf = pd.read_csv(curFilePath,delimiter=';',dtype=str)

    casedatadf['Antal_bekræftede_COVID-19'] = pd.to_numeric(casedatadf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))

    dfCase.loc[curIndex] = casedatadf['Antal_bekræftede_COVID-19'].values

# Data: Collect the data in same style

In [4]:
# Same things, but with cases
dfCaseDiff = dfCase.diff().iloc[1:]

dfCaseDiff['Dato'] = curDates[1:]

# Bad hack to ignore 10-15 year olds!
dfCaseDiff['10-19'] = np.multiply(dfCaseDiff['10-19'],0.4)

dfCaseDiffSum = dfCaseDiff.iloc[:,1:-2].cumsum(axis=1)
dfCaseDiffSum['Dato'] = curDates[1:]

# dfCaseDiff = dfCaseDiff.iloc[:-1]
# dfCaseDiffSum = dfCaseDiffSum.iloc[:-1]


dfCaseDiffSum['I alt'] = dfCaseDiffSum['90+']
# dfCaseDiffSum

In [5]:
dfAdmDiff = dfAdm.diff().iloc[1:]

dfAdmDiff['Dato'] = curDates[1:]

dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
dfAdmDiffSum['Dato'] = curDates[1:]

# dfAdmDiff = dfAdmDiff.iloc[:-1]
# dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']

# dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

# Load calculations

In [6]:
datafilename = '../model_results_spread_prediction_var_vacc.csv'
df = pd.read_csv(datafilename)
df.End =  pd.to_datetime(df.End,format='%Y-%m-%d')

# Add the RT2 columns
df['RT2'] = np.round(np.multiply(df.RT,10)).astype(int)

# Only take the vaccination-effeciency of 0.75
curdf = df[df.vacc1 == 0.75]

In [7]:
# Also load the delayed one
datafilenameD = '../model_results_spread_prediction_var_vacc_delayed.csv'
dfD = pd.read_csv(datafilenameD)
dfD.End =  pd.to_datetime(dfD.End,format='%Y-%m-%d')

# Add the RT2 columns
dfD['RT2'] = np.round(np.multiply(dfD.RT,10)).astype(int)

# Only take the vaccination-effeciency of 0.75
curdfD = dfD[dfD.vacc1 == 0.75]

In [8]:
# curdf[curdf.Age == '30-39'] #.End.iloc[-10:]

# df

# Nye smittetilfælde

In [9]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

casedf = pd.DataFrame()

casedf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 21
weekNums = np.arange(firstWeek,firstWeek+numVals)
casedf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


casedf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    casedf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Count.values 
    curVals = prevVals + newVals
    # casedf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Count.values 
    casedf[curAge] = curVals
    prevVals = curVals

    casedf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Count.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        casedf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 
        casedf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 


# # Remove everythings after september first
# casedf = casedf[casedf.Date <= np.datetime64('2021-09-01')]

# casedf

In [10]:
curSumTotal = int(np.round(casedf.Total_12.sum())) # All weeks
curSumTotal = int(np.round(casedf.Total_12[1:].sum())) # Excluding the current weeks (i.e. data)
# casedf
curSumTotal

61559

In [11]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
# curMaxY = 230000
# firstDate = np.datetime64('2021-05-18')
firstDate = np.datetime64('2021-05-25')
predDayTest = '25. maj'
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False


plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumTotal),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }

    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("cases.html",title='Nye smittetilfælde')

show(column(plot,slider,divBelow))

None


In [12]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)
sourceData = ColumnDataSource(dfCaseDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
# curMaxY = 130000
firstDate = np.datetime64('2021-05-18')
predDayTest = '25. maj'
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,None),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False


plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumTotal),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumTotal),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    

    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("casesData.html",title='Nye smittetilfælde')

show(column(plot,slider,divBelow))

None


In [13]:
# Same figure, but thinner

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)
sourceData = ColumnDataSource(dfCaseDiffSum)

figW = 650
figH = 500

curMaxY = 1.1*max(casedf.Total_17)
# curMaxY = 130000
firstDate = np.datetime64('2021-05-18')
predDayTest = '25. maj'
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,None),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False


plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumTotal),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumTotal),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    

    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("casesDataThin.html",title='Nye smittetilfælde')

show(column(plot,slider,divBelow))

None


# Hospitaliseringer

In [14]:
curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)
# curdf['Hosp'] = np.multiply(np.multiply(curdf.Count,curdf.CaseHospRate),0.75)
# curdf[curdf.Age == 'l0-19'].CaseHospRate.iloc[0]

<ipython-input-14-46103b8eeb16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)


In [15]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

admdf = pd.DataFrame()

admdf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 21
weekNums = np.arange(firstWeek,firstWeek+numVals)
admdf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


admdf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Hosp.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Hosp.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    admdf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Hosp.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Hosp.values 
    curVals = prevVals + newVals
    # admdf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Hosp.values 
    admdf[curAge] = curVals
    prevVals = curVals

    admdf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Hosp.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        admdf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 
        admdf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 


# # Remove everythings after september first
# admdf = admdf[admdf.Date <= np.datetime64('2021-09-01')]

# admdf

In [16]:
curSumAdm = int(np.round(admdf.Total_12.sum())) # All weeks
print(curSumAdm)
curSumAdm = int(np.round(admdf.Total_12[1:].sum())) # Excluding the current weeks (i.e. data)
curSumAdm

1704


1501

In [17]:
# dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
# dfAdm = dfAdm.transpose()
# dfAdm.columns = dfAdm.iloc[0]
# dfAdm = dfAdm.drop(['Aldersgruppe']) 
# curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')


# dfAdmDiff = dfAdm.diff().iloc[1:]

# dfAdmDiff['Dato'] = curDates[1:]

# dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
# dfAdmDiffSum['Dato'] = curDates[1:]

# dfAdmDiff = dfAdmDiff.iloc[:-1]
# dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


# dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']
# dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

In [18]:

from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 30000
# curMaxY = 600

firstDate = np.datetime64('2021-05-18')
firstDate = np.datetime64('2021-05-25')
predDayTest = '25. maj'
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False


# firstDate = np.datetime64('2021-04-01')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



# for k in range(0,len(allAges)):
#     curAge = allAges[k]
#     curColor = cmaptohex(cmap(k))
#     plot.line('Dato', curAge, source=sourceData,color=curColor)
#     plot.circle('Dato', curAge, source=sourceData,color=curColor)

# plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
# plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumAdm),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumAdm),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    

    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hosp.html",title='Hospitaliseringer')

show(column(plot,slider,divBelow))

None


In [19]:

from bokeh.models import DataRange1d
from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

# curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 600


# firstDate = np.datetime64('2021-05-18')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

firstDate = np.datetime64('2021-04-01')
predDayTest = '25. maj'
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False


plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumAdm),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumAdm),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hospData.html",title='Hospitaliseringer')

show(column(plot,slider,divBelow))

None


In [20]:
# Same, but thin

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 650
figH = 500

# curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 600


# firstDate = np.datetime64('2021-05-18')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

firstDate = np.datetime64('2021-04-01')
predDayTest = '25. maj'
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False


plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumAdm),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumAdm),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hospDataThin.html",title='Hospitaliseringer')

show(column(plot,slider,divBelow))

None


# Nye smittetilfælde, delayed

In [21]:

# curdfD['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

casedf = pd.DataFrame()

casedf['Date'] = curdfD.End.unique().astype('datetime64[D]')
numVals = len(curdfD.End.unique())
firstWeek = 21
weekNums = np.arange(firstWeek,firstWeek+numVals)
casedf['Week'] = weekNums

 


allAges = dfD.Age.unique()

allRTs = dfD.RT2.unique()


casedf['Total'] = np.zeros(curdfD[(curdfD.Age == '20-29') & (curdfD.RT2 == 12)].Count.values.shape)
prevVals = np.zeros(curdfD[(curdfD.Age == '20-29') & (curdfD.RT2 == 12)].Count.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    casedf[curColName] = np.zeros(curdfD[(curdfD.Age == '20-29') & (curdfD.RT2 == 12)].Count.values.shape)



for curAge in allAges:
    newVals = curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == 12)].Count.values 
    curVals = prevVals + newVals
    # casedf[curAge] = curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == 12)].Count.values 
    casedf[curAge] = curVals
    prevVals = curVals

    casedf['Total'] += curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == 12)].Count.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        casedf[curColName] = curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == curRT)].Count.values 
        casedf['Total_'+str(curRT)] += curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == curRT)].Count.values 


# # Remove everythings after september first
# casedf = casedf[casedf.Date <= np.datetime64('2021-09-01')]

# casedf
curSumTotal = int(np.round(casedf.Total_12.sum())) # All weeks
curSumTotal = int(np.round(casedf.Total_12[1:].sum())) # Excluding the current weeks (i.e. data)
# casedf
curSumTotal


30416

In [22]:
dfCaseDiffSumTest = dfCaseDiffSum

extraRow = dfCaseDiffSumTest.iloc[len(dfCaseDiffSum)-1]
extraRow.Dato = extraRow.Dato + np.timedelta64(7,'D')
dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)
dfCaseDiffSumTest = dfCaseDiffSumTest.append(extraRow)
dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)


extraRow = dfCaseDiffSumTest.iloc[len(dfCaseDiffSumTest)-1]
extraRow.Dato = extraRow.Dato + np.timedelta64(7,'D')
dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)
dfCaseDiffSumTest = dfCaseDiffSumTest.append(extraRow)
dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)

dfCaseDiffSumTest = dfCaseDiffSumTest.iloc[-3:]
# dfCaseDiffSumTest

# dfCaseDiffSumTest.Dato = pd.to_datetime(dfCaseDiffSumTest.Dato)
# dfCaseDiffSumTest.iloc[-1].Dato = dfCaseDiffSumTest.iloc[-2].Dato + np.timedelta64(14,'D')

 
# dfCaseDiffSumTest
# # extraRow
# dfCaseDiffSum.loc[len(dfCaseDiffSum)] = extraRow
# dfCaseDiffSum.iloc[len(dfCaseDiffSum)] = extraRow
# # dfCaseDiffSum.loc[dfCaseDiffSum.index.repeat(2)]
# # dfCaseDiffSum.iloc[-1+1] = extraRow 
# # dfCaseDiffSum
# # dfCaseDiffSum.append(pd.DataFrame(extraRow,columns=dfCaseDiffSum.columns),ignore_index=True)
# # dfCaseDiffSum.append(extraRow,ignore_index=True)

C:\ProgramData\Anaconda3\envs\main\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [23]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)
sourceData = ColumnDataSource(dfCaseDiffSum)
sourceDataDel = ColumnDataSource(dfCaseDiffSumTest)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
# curMaxY = 130000
firstDate = np.datetime64('2021-05-18')
predDayTest = '25. maj'
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,None),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False


plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge,line_dash="4 4", source=sourceDataDel,color=curColor)
    plot.triangle('Dato', curAge, source=sourceDataDel,color=curColor)

plot.triangle('Dato', 'I alt', source=sourceDataDel,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceDataDel,line_dash="4 4", line_width=3, line_alpha=0.6,color='black')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumTotal),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumTotal),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    

    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("casesDataDelayed.html",title='Nye smittetilfælde, forsinket')

show(column(plot,slider,divBelow))

None


In [24]:
# The same, but thin

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)
sourceData = ColumnDataSource(dfCaseDiffSum)
sourceDataDel = ColumnDataSource(dfCaseDiffSumTest)

figW = 650
figH = 500

curMaxY = 1.1*max(casedf.Total_17)
# curMaxY = 130000
firstDate = np.datetime64('2021-05-18')
predDayTest = '25. maj'
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,None),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False


plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge,line_dash="4 4", source=sourceDataDel,color=curColor)
    plot.triangle('Dato', curAge, source=sourceDataDel,color=curColor)

plot.triangle('Dato', 'I alt', source=sourceDataDel,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceDataDel,line_dash="4 4", line_width=3, line_alpha=0.6,color='black')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumTotal),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumTotal),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    

    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("casesDataDelayedThin.html",title='Nye smittetilfælde, forsinket')

show(column(plot,slider,divBelow))

None


# Delayed different number of weeks

In [25]:
datafilenameD2 = '../compare_model_results.csv'
dfD2 = pd.read_csv(datafilenameD2)
dfD2.End =  pd.to_datetime(dfD2.End,format='%Y-%m-%d')
dfD2.Start =  pd.to_datetime(dfD2.Start,format='%Y-%m-%d')
dfD2['RT2'] = np.round(np.multiply(dfD2.RT,10)).astype(int)
# dfD['Hosp'] = np.multiply(dfD.Count,dfD.CaseHospRate)

In [26]:
dfD2

,Unnamed: 0,Start,End,RT,Count,CountCumul,HospCumul,RT2
0,0,2021-06-01,2021-06-01,0.9,5591.000000,0.000000,0.000000,9
1,1,2021-06-01,2021-06-01,1.0,5591.000000,0.000000,0.000000,10
2,2,2021-06-01,2021-06-01,1.1,5591.000000,0.000000,0.000000,11
3,3,2021-06-01,2021-06-01,1.2,5591.000000,0.000000,0.000000,12
4,4,2021-06-01,2021-06-01,1.3,5591.000000,0.000000,0.000000,13
...,...,...,...,...,...,...,...,...
751,751,2021-07-13,2021-09-07,1.3,0.069211,5610.659559,153.486523,13
752,752,2021-07-13,2021-09-07,1.4,0.165498,6935.989443,191.583393,14
753,753,2021-07-13,2021-09-07,1.5,0.371415,8560.189692,238.829715,15
754,754,2021-07-13,2021-09-07,1.6,0.788037,10554.667444,297.514015,16


In [27]:
deldf = pd.DataFrame()

deldf['Date'] = dfD2.End.unique().astype('datetime64[D]')
# numVals = len(dfD2.End.unique())
# firstWeek = 21
# weekNums = np.arange(firstWeek,firstWeek+numVals)
# deldf['Week'] = weekNums

defRT2 = 12


# defInit = dfD2.Start.iloc[0]
defInit = dfD2.Start.unique()[0]
defVals = dfD2[(dfD2.RT2==defRT2) & (dfD2.Start == defInit)].Count.values
deldf['ToShow0'] = defVals

defInit = dfD2.Start.unique()[1]
defVals = dfD2[(dfD2.RT2==defRT2) & (dfD2.Start == defInit)].Count.values
while (len(defVals) < len(deldf)):
    defVals = np.append(np.nan,defVals)
deldf['ToShow1'] = defVals

defInit = dfD2.Start.unique()[2]
defVals = dfD2[(dfD2.RT2==defRT2) & (dfD2.Start == defInit)].Count.values
while (len(defVals) < len(deldf)):
    defVals = np.append(np.nan,defVals)
deldf['ToShow2'] = defVals

defInit = dfD2.Start.unique()[3]
defVals = dfD2[(dfD2.RT2==defRT2) & (dfD2.Start == defInit)].Count.values
while (len(defVals) < len(deldf)):
    defVals = np.append(np.nan,defVals)
deldf['ToShow3'] = defVals


# # prevVals = np.zeros(deldf['ToShow'].values.shape)


# for curRT in allRTs:
#     curColName = 'Total_'+str(curRT)


#     deldf[curColName] = dfD2[dfD2.RT2 == curRT].Count.values


numWeeksToShow = 4

posStart = dfD2.Start.unique()

for k in range(numWeeksToShow):
    curStart = posStart[k]
    
    for curRT in allRTs:
        curColName = 'Week'+str(k)+'RT'+str(curRT)
        # print(curColName)

        curVals = dfD2[(dfD2.Start == curStart) & (dfD2.RT2 == curRT)].Count.values

        # print(len(curVals))

        while (len(curVals) < len(deldf)):
            # curVals = np.append([])
            curVals = np.append(np.nan,curVals)
            # print(curVals)

        deldf[curColName] = curVals
        # deldf[curColName] = dfD2[(dfD2.Start == curStart) & (dfD2.RT2 == curRT)].Count.values


deldf

,Date,ToShow0,ToShow1,ToShow2,ToShow3,Week0RT9,Week0RT10,Week0RT11,Week0RT12,Week0RT13,...,Week2RT17,Week3RT9,Week3RT10,Week3RT11,Week3RT12,Week3RT13,Week3RT14,Week3RT15,Week3RT16,Week3RT17
0,2021-06-01,5591.000000,NaN,NaN,NaN,5591.000000,5591.000000,5591.000000,5591.000000,5591.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-06-08,6236.359617,5591.000000,NaN,NaN,4063.050135,4753.370157,5478.356816,6236.359617,7025.941031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-06-15,7046.051663,5616.383649,5591.000000,NaN,2990.803528,4093.424574,5437.310727,7046.051663,8943.191341,...,5591.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-06-22,7890.424594,6137.298021,5094.874956,5591.000000,2186.551451,3498.836008,5352.672413,7890.424594,11274.423772,...,8559.054899,5591.000000,5591.000000,5591.000000,5591.000000,5591.000000,5591.000000,5591.000000,5591.000000,5591.000000
4,2021-06-29,8877.055533,6914.015570,5477.044683,4626.905097,1613.499009,3014.717805,5303.629292,8877.055533,14248.333012,...,15457.200802,3014.474555,3526.639572,4064.524600,4626.905097,5212.714527,5821.013096,6450.964526,7101.818374,7772.896308
5,2021-07-06,9152.144710,7184.885998,5627.860996,4456.394981,1098.950373,2393.670447,4831.256973,9152.144710,16429.779680,...,26574.497784,1891.584461,2588.955858,3438.919468,4456.394981,5656.273175,7053.418386,8662.670594,10498.847205,12576.744588
6,2021-07-13,8504.215761,6742.776849,5291.080800,4130.793886,679.963017,1724.203810,3982.783479,8504.215761,16964.214568,...,41266.919606,1144.526514,1831.517174,2802.079290,4130.793886,5902.716324,8213.766475,11171.094391,14893.423782,19511.375179
7,2021-07-20,6609.840422,5296.156392,4184.305930,3271.534297,354.308463,1045.153672,2757.643699,6609.840422,14538.051475,...,52493.934652,593.510256,1109.541473,1953.184816,3271.534297,5255.345245,8146.280106,12244.405152,17915.828550,25600.355448
8,2021-07-27,4187.207598,3391.299124,2701.950569,2125.400629,151.410099,519.420662,1562.982460,4187.207598,10054.482982,...,51789.193399,253.379633,552.942555,1118.673341,2125.400629,3829.308488,6591.772312,10905.428756,17420.966604,26972.278805
9,2021-08-03,2082.037131,1706.993840,1373.246819,1088.475147,51.217541,204.242549,699.466198,2082.037131,5379.479409,...,36324.701753,85.635420,217.986310,506.174025,1088.475147,2192.020961,4168.856467,7534.475111,12999.910257,21480.792286


In [28]:
startDayStrs = []
startDayStrs.append('Fra '+pd.to_datetime(deldf.Date[0]).strftime('%d. %b'))
startDayStrs.append('Fra '+pd.to_datetime(deldf.Date[1]).strftime('%d. %b'))
startDayStrs.append('Fra '+pd.to_datetime(deldf.Date[2]).strftime('%d. %b'))
startDayStrs.append('Fra '+pd.to_datetime(deldf.Date[3]).strftime('%d. %b'))
startDayStrs

['Fra 01. jun', 'Fra 08. jun', 'Fra 15. jun', 'Fra 22. jun']

In [29]:
dfCaseDiffSumTest = dfCaseDiffSum

for k in range(numWeeksToShow):
    extraRow = dfCaseDiffSumTest.iloc[len(dfCaseDiffSum)-1]
    extraRow.Dato = extraRow.Dato + np.timedelta64(k*7,'D')
    dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)
    dfCaseDiffSumTest = dfCaseDiffSumTest.append(extraRow)
    dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)


dfCaseDiffSumTest = dfCaseDiffSumTest.iloc[-numWeeksToShow:]
# dfCaseDiffSumTest

C:\ProgramData\Anaconda3\envs\main\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [30]:
cmap = plt.cm.get_cmap('plasma',numWeeksToShow+1)

source = ColumnDataSource(deldf)
sourceData = ColumnDataSource(dfCaseDiffSum)
sourceDataDel = ColumnDataSource(dfCaseDiffSumTest)


figW = 1000
figH = 600

# firstDate = np.datetime64('2021-05-18')
# predDayTest = '25. maj'
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",months="%d. %b",hours="%d. %b",minutes="%d. %b")


for k in range(numWeeksToShow):
    curColor = cmaptohex(cmap(k))

    plot.line('Date', 'ToShow'+str(k), source=source, line_width=3, line_alpha=0.6,color=curColor,legend_label=startDayStrs[k])


plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


plot.triangle('Dato', 'I alt', source=sourceDataDel,color='black',legend_label='(Data som 1. juni)')
plot.line('Dato', 'I alt', source=sourceDataDel,line_dash="4 4", line_width=3, line_alpha=0.6,color='black')



minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

# update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y0 = data['ToShow0']
    var y1 = data['ToShow1']
    var y2 = data['ToShow2']
    var y3 = data['ToShow3']

    const curName0 = 'Week0RT'+curRT; 
    const curName1 = 'Week1RT'+curRT; 
    const curName2 = 'Week2RT'+curRT; 
    const curName3 = 'Week3RT'+curRT; 

    for (var i = 0; i < y0.length; i++) {
        y0[i] = data[curName0][i]
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i]
        y3[i] = data[curName3][i]
    }

    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

    # // Summing up
    # for (var i = 0; i < y1.length; i++) {
    #     yT[i] = data[curNameT][i]
    # }

    # var totSum = yT.slice(1).reduce((a, b) => a + b)
    # divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    

output_file("casesDataDelayedMulti.html",title='Nye smittetilfælde, forsinket')

show(column(plot,slider))

In [31]:
# The same, but thin

cmap = plt.cm.get_cmap('plasma',numWeeksToShow+1)

source = ColumnDataSource(deldf)
sourceData = ColumnDataSource(dfCaseDiffSum)
sourceDataDel = ColumnDataSource(dfCaseDiffSumTest)


figW = 650
figH = 500

# firstDate = np.datetime64('2021-05-18')
# predDayTest = '25. maj'
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
plot.y_range.start = 0
plot.left[0].formatter.use_scientific = False

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",months="%d. %b",hours="%d. %b",minutes="%d. %b")


for k in range(numWeeksToShow):
    curColor = cmaptohex(cmap(k))

    plot.line('Date', 'ToShow'+str(k), source=source, line_width=3, line_alpha=0.6,color=curColor,legend_label=startDayStrs[k])


plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


plot.triangle('Dato', 'I alt', source=sourceDataDel,color='black',legend_label='(Data som 1. juni)')
plot.line('Dato', 'I alt', source=sourceDataDel,line_dash="4 4", line_width=3, line_alpha=0.6,color='black')



minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

# update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y0 = data['ToShow0']
    var y1 = data['ToShow1']
    var y2 = data['ToShow2']
    var y3 = data['ToShow3']

    const curName0 = 'Week0RT'+curRT; 
    const curName1 = 'Week1RT'+curRT; 
    const curName2 = 'Week2RT'+curRT; 
    const curName3 = 'Week3RT'+curRT; 

    for (var i = 0; i < y0.length; i++) {
        y0[i] = data[curName0][i]
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i]
        y3[i] = data[curName3][i]
    }

    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

    

output_file("casesDataDelayedMultiThin.html",title='Nye smittetilfælde, forsinket')

show(column(plot,slider))

In [32]:

# # x = [x*0.005 for x in range(0, 201)] 

# # source = ColumnDataSource(data=dict(x=x, y=x))
# # source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
# source = ColumnDataSource(casedf)
# sourceData = ColumnDataSource(dfCaseDiffSum)
# sourceDataDel = ColumnDataSource(dfCaseDiffSumTest)

# figW = 1000
# figH = 600

# # curMaxY = 1.1*max(casedf.Total_17)
# # curMaxY = 130000
# firstDate = np.datetime64('2021-05-18')
# predDayTest = '25. maj'
# firstDate = np.datetime64('2021-04-01')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
# # plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,None),tools=[])
# plot.y_range.start = 0

# plot.toolbar.logo = None
# plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

# # x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# # x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

# plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
# months="%d. %b",
# hours="%d. %b",
# minutes="%d. %b")

# # plot.xaxis.ticker = x_values_timestamp
# # plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# # plot.y_range.bounds = (0,None)
# cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


# curAge = allAges[0]
# curColor = cmaptohex(cmap(0))
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# # plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# # plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

# for k in range(1,len(allAges)):
#     curAge = allAges[k]
#     prevAge = allAges[k-1]
#     curColor = cmaptohex(cmap(k))
#     # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
#     plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


# plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



# # for k in range(0,len(allAges)):
# #     curAge = allAges[k]
# #     curColor = cmaptohex(cmap(k))
# #     plot.line('Dato', curAge, source=sourceData,color=curColor)
# #     plot.circle('Dato', curAge, source=sourceData,color=curColor)

# plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
# plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


# # for k in range(0,len(allAges)):
# #     curAge = allAges[k]
# #     curColor = cmaptohex(cmap(k))
# #     plot.line('Dato', curAge,line_dash="4 4", source=sourceDataDel,color=curColor)
# #     plot.triangle('Dato', curAge, source=sourceDataDel,color=curColor)

# plot.triangle('Dato', 'I alt', source=sourceDataDel,color='black',legend_label='Data')
# plot.line('Dato', 'I alt', source=sourceDataDel,line_dash="4 4", line_width=3, line_alpha=0.6,color='black')


# # testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# # testData = np.multiply(np.ones((len(testDates),)),5000)
# # plot.line(testDates,testData)

# minRT = 0.9
# maxRT = 1.7
# slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

# divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumTotal),width=200, height=100)
# # divBelow = Div(text="Samlet over hele perioden: "+str(curSumTotal),width=200, height=100)

# update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
#     var data = source.data;
#     var f = slider.value;
#     var curRT = Math.round(f*10)
#     var yT = data['Total']

#     const curNameT = 'Total_'+curRT; 

#     // Summing up
#     for (var i = 0; i < y1.length; i++) {
#         yT[i] = data[curNameT][i]
#     }
    

#     var totSum = yT.slice(1).reduce((a, b) => a + b)
#     divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
#     // necessary becasue we mutated source.data in-place
#     source.change.emit();
# """)
# slider.js_on_change('value', update_curve)
# slider.js_on_change('start', update_curve)

# print(plot.y_range.bounds)
 
# output_file("casesDataDelayedMulti.html",title='Nye smittetilfælde, forsinket')

# show(column(plot,slider,divBelow))

In [33]:
# casedf = pd.DataFrame()

# casedf['Date'] = dfD2.End.unique().astype('datetime64[D]')
# numVals = len(curdfD.End.unique())
# firstWeek = 21
# weekNums = np.arange(firstWeek,firstWeek+numVals)
# casedf['Week'] = weekNums

 


# allAges = dfD.Age.unique()

# allRTs = dfD.RT2.unique()


# casedf['Total'] = np.zeros(curdfD[(curdfD.Age == '20-29') & (curdfD.RT2 == 12)].Count.values.shape)
# prevVals = np.zeros(curdfD[(curdfD.Age == '20-29') & (curdfD.RT2 == 12)].Count.values.shape)

# for curRT in allRTs:
#     curColName = 'Total_'+str(curRT)
#     casedf[curColName] = np.zeros(curdfD[(curdfD.Age == '20-29') & (curdfD.RT2 == 12)].Count.values.shape)



# for curAge in allAges:
#     newVals = curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == 12)].Count.values 
#     curVals = prevVals + newVals
#     # casedf[curAge] = curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == 12)].Count.values 
#     casedf[curAge] = curVals
#     prevVals = curVals

#     casedf['Total'] += curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == 12)].Count.values 

#     for curRT in allRTs:
#         curColName = curAge+'_'+str(curRT)
#         casedf[curColName] = curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == curRT)].Count.values 
#         casedf['Total_'+str(curRT)] += curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == curRT)].Count.values 


# # # Remove everythings after september first
# # casedf = casedf[casedf.Date <= np.datetime64('2021-09-01')]

# # casedf
# curSumTotal = int(np.round(casedf.Total_12.sum())) # All weeks
# curSumTotal = int(np.round(casedf.Total_12[1:].sum())) # Excluding the current weeks (i.e. data)
# # casedf
# curSumTotal


# OLD

In [34]:

# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color='#00ff00')
# plot.line('Week', '20-29', source=source, line_width=3, line_alpha=0.6, color='#20dd20')
# plot.line('Week', '30-39', source=source, line_width=3, line_alpha=0.6, color='#307030')
# plot.line('Week', '40-49', source=source, line_width=3, line_alpha=0.6, color='#406040')
# plot.line('Week', '50-59', source=source, line_width=3, line_alpha=0.6, color='#505050')
# plot.line('Week', '60-69', source=source, line_width=3, line_alpha=0.6, color='#604060')
# plot.line('Week', '70-79', source=source, line_width=3, line_alpha=0.6, color='#703070')
# plot.line('Week', '80-89', source=source, line_width=3, line_alpha=0.6, color='#802080')
# plot.line('Week',   '90+', source=source, line_width=3, line_alpha=0.6, color='#901090')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6,color='black')

# plot.circle('Week', '10-19', source=source, color='#108000')
# plot.circle('Week', '20-29', source=source, color='#208000')
# plot.circle('Week', '30-39', source=source, color='#308000')
# plot.circle('Week', '40-49', source=source, color='#408000')
# plot.circle('Week', '50-59', source=source, color='#508000')
# plot.circle('Week', '60-69', source=source, color='#608000')
# plot.circle('Week', '70-79', source=source, color='#708000')
# plot.circle('Week', '80-89', source=source, color='#808000')
# plot.circle('Week',   '90+', source=source, color='#908000')
# plot.circle('Week', 'Total', source=source,color='black')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6)


# plot.varea('Week', 0 ,'10-19', source=source, fill_color=cmaptohex(cmap(1)))
# plot.varea('Week','10-19', '20-29', source=source, color=cmaptohex(cmap(2)))
# plot.varea('Week','20-29', '30-39', source=source, color=cmaptohex(cmap(3)))
# plot.varea('Week','30-39', '40-49', source=source, color=cmaptohex(cmap(4)))
# plot.varea('Week','40-49', '50-59', source=source, color=cmaptohex(cmap(5)))
# plot.varea('Week','50-59', '60-69', source=source, color=cmaptohex(cmap(6)))
# plot.varea('Week','60-69', '70-79', source=source, color=cmaptohex(cmap(7)))
# plot.varea('Week','70-79', '80-89', source=source, color=cmaptohex(cmap(8)))
# plot.varea('Week','80-89',   '90+', source=source, color=cmaptohex(cmap(9)))

# slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

In [35]:
# curdf['RT2'] = np.round(np.multiply(curdf.RT,10)).astype(int)


    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i]
    #     y3[i] = data[curName3][i]
    #     y4[i] = data[curName4][i]
    #     y5[i] = data[curName5][i]
    #     y6[i] = data[curName6][i]
    #     y7[i] = data[curName7][i]
    #     y8[i] = data[curName8][i]
    #     y9[i] = data[curName9][i]
    #     yT[i] = data[curNameT][i]
    # }


    # // Summing up
    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i] + y1[i]
    #     y3[i] = data[curName3][i] + y2[i]
    #     y4[i] = data[curName4][i] + y3[i]
    #     y5[i] = data[curName5][i] + y4[i]
    #     y6[i] = data[curName6][i] + y5[i]
    #     y7[i] = data[curName7][i] + y6[i]
    #     y8[i] = data[curName8][i] + y7[i]
    #     y9[i] = data[curName9][i] + y8[i]
    #     yT[i] = data[curNameT][i]
    # }

In [36]:

# # x = [x*0.005 for x in range(0, 201)] 

# # source = ColumnDataSource(data=dict(x=x, y=x))
# # source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
# source = ColumnDataSource(curdf)

# plot = figure(plot_width=400, plot_height=400)
# plot.line('x', 'Count', source=source, line_width=3, line_alpha=0.6)

# # slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")
# minRT = 0.9
# maxRT = 1.7
# slider = Slider(start=minRT, end=maxRT, value=1.1, step=.1, title="RT")

# update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
#     var data = source.data;
#     var f = slider.value;
#     var age = data['Age']
#     var RT = data['RT2']
#     var End = data['End']
#     var Count = data['Count']

#     var x = data['x']
#     var y = data['y']

#     console.log(Count)
#     for (var i = 0; i < RT.length; i++) {
#         y[i] = 6
#         Count[i] = 0
        
#         if (RT[i] == Math.round(f*10)){
#             y[i] = data['Count'][i]
#             console.log(data['Count'][i])
#             Count[i] = data['Count'][i]

#             // End.push(data['End'][i])
#             // Count.push(data['Count'][i])
#         }

#     }

#     //for (var i = 0; i < x.length; i++) {
#         // y[i] = Math.pow(x[i], f)
#     //}
    
#     // necessary becasue we mutated source.data in-place
#     source.change.emit();
# """)
# slider.js_on_change('value', update_curve)

# output_file("plot.html")

# show(column(slider, plot))

In [37]:

# testdf['Total'] = curdf[(curdf.RT2 == 11)].Count.values 
# for curRT in allRTs:
#     curColName = 'Total_'+str(curRT)
#     testdf[curColName] = curdf[(curdf.RT2 == curRT)].Count.values 

# testdf['20_RT10'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 10)].Count.values
# testdf['20_RT11'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['20_RT12'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['20_RT13'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['20_RT14'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values
# testdf['RT11_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['RT12_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['RT13_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['RT14_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values

    # if (Math.round(f*10) == 11){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT11_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 12){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT12_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 13){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT13_20'][i]
    #     }
    # }
    
